# Project - To implement the DEX (Deep Expectation) Network used for age estimation
<hr/>

##By:

|||
|-|-|
| <font size="2">Name</font> | <font size="4">Ajay Dabas</font> |
| <font size="2">Roll Number</font> | <font size="4">2K17/SE/12</font> |
| <font size="2">Branch</font> | <font size="4">Software Engineering</font> |
| <font size="2">Subject</font> | <font size="4">Computer Vision</font> |
| <font size="2">Faculty</font> | <font size="4">Dr. Rajiv Kapoor</font> |

## Table of Contents
<hr/>

1. [Summary](#scrollTo=Tdt8Zg-6V7Zh)
2. [Importing Python Modules](#scrollTo=H4hPfGGtV5N0)
3. [Dataset Preprocessing](#scrollTo=v5b_mbfN4Eq3)
4. [Model Training](#scrollTo=-9sXLbK8-Xph)
5. [Model Testing & Results](#scrollTo=Ki1h9GSnU3GS)
6. [References](#scrollTo=wnhzkTURWk-r)

# Summary
<hr/>

The estimation of apparent age in still face images with deep learning uses the VGG-19 architecture. The age regression problem is posed as a deep classification problem followed by a softmax expected value refinement and show improvements over direct regression training of CNNs. Deep EXpectation (DEX) of apparent age, uses an ensemble of 3 (Original paper uses 20) VGG19 networks to predict the age.

# Importing Python Modules

In [0]:
import os
import re
import PIL
import tarfile
import numpy as np
from datetime import datetime
from shutil import copy as copy_file
from tqdm import tqdm_notebook as tqdm
from IPython.display import Image, display
from keras.models import Model, load_model
from keras.layers import Dense, Flatten
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
# Constants
imgs_folder_path = "dataset/faces/images"

# Dataset Preprocessing

In [0]:
# mkdir dataset
# ls dataset
# !wget "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar"
# cd ..

In [0]:
%%time
# This cell takes approx. "9 min" to execute
with tarfile.open('dataset/wiki_crop.tar') as tar:
  tar.extractall(path='dataset/faces') # Extract all images from .tar file

In [0]:
%%time
# This cell takes approx. "49.3 sec" to execute
exp_count = 0
for _rootdir, _, _files in tqdm(os.walk('dataset/faces/wiki_crop')):
  for _file in _files:
    try: # Move all files(face images) into single folder named `all`
      os.rename(os.path.join(_rootdir, _file),os.path.join(imgs_folder_path, _file))
    except:  # If any error, continue
      exp_count+=1
      continue
print("Exceptions encountered: {}".format(exp_count))

In [0]:
print('Number of images: {}'.format(len(os.listdir(imgs_folder_path))))

In [0]:
%%time
# This cell takes approx. "1min 10s" mins to execute
for _file in tqdm(os.listdir(imgs_folder_path)):
  try:
    # Example:- 23300_1962-06-19_2011.jpg --> Split: ["23300", "1964-06-19", "2011.jpg"]
    file_name = _file.split("_")
    # Date of birth of the person
    begin_date = datetime.strptime(file_name[1], '%Y-%m-%d')
    # The year the picture was taken - 2011 + assuming a mid-year date of month=7 & day=1 for when the photo was taken.
    end_date = datetime(year=int(file_name[2][:4]), month=7, day=1)
    age = end_date.year - begin_date.year
    if age >= 10 and age < 100: # Select only people with age in the range [10,100)
      # Rename face images file names with age
      os.rename(os.path.join(imgs_folder_path, _file), os.path.join(imgs_folder_path, str(age) + "_" + file_name[0] + '.jpg'))
    else:
      # If invalid age, remove the image
      os.remove(os.path.join(imgs_folder_path, _file))
  except:
      # If any error, remove the image
      os.remove(os.path.join(imgs_folder_path, _file))

In [0]:
print('Number of images: {}'.format(len(os.listdir(imgs_folder_path))))

In [0]:
num_samples = 2
random_choice = list(np.random.choice(os.listdir(imgs_folder_path),num_samples))
samples = []
for choice in random_choice:
  space = '='*5
  print('\n',space,'\tAge: {}\t'.format(choice.split('_')[0]),space)
  display(Image(filename=os.path.join(imgs_folder_path,choice), width=200, height=200))

# Parameters & Helper functions for Training & Testing

In [0]:
epochs = 2
batch_size = 32
n_models = 3 # Number of VGG19 models in the ensemble
validation_size = 0.5

In [0]:
image_paths_all = os.listdir(imgs_folder_path) # all images
image_paths_train = image_paths_all[:int(len(image_paths_all)*validation_size)] # 50% train
image_paths_val = image_paths_all[int(len(image_paths_all)*validation_size):] # 50% validation
print("Training on {} images".format(len(image_paths_train)))
print("Validating on {} images".format(len(image_paths_val)))

In [0]:
# Preprocessing image before feeding to model
def preprocess(img_paths):
  images = []
  labels = []
  for img_path in img_paths:
    image = load_img(os.path.join(imgs_folder_path,img_path), target_size=(224, 224)) # Load image
    image = img_to_array(image) # Convert to numpy array
    image = preprocess_input(image) # Preprocess image as required by VGG19 model
    if(image.shape!=(224,224,3)):
      continue
    images.append(image)
    labels.append(float(img_path.split('_')[0])) # Extract label from image filename
  return np.concatenate([images]), np.array(labels).reshape(len(labels),1)
# Generator for batch training
def batch_generator(img_paths, batch_size=16, seed=1):
  np.random.seed(seed)
  np.random.shuffle(img_paths) # Shuffle before use
  i=0
  while True:
    if(i>=len(img_paths)): # Resetting generator
      i=0
    images, labels = preprocess(img_paths[i:i+batch_size])
    i += batch_size
    yield images, labels

# Model Training

In [0]:
def get_model(train_last_layers=4):
  base_model = VGG19(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
  # Freeze the layers except the last `train_last_layers` layers
  for layer in base_model.layers[:-train_last_layers]:
    layer.trainable = False
  # Check the trainable status of the individual layers
  top_model = Flatten()(base_model.output)
  top_model = Dense(1, activation='relu')(top_model)
  model = Model(inputs=base_model.inputs, outputs=top_model)
  model.compile(optimizer='Adam',loss='mean_absolute_error')
  return model

In [0]:
# for layer in model.layers:
#     print(layer, layer.trainable)

In [0]:
# model.summary()

In [0]:
for model_number in range(n_models):
  model = None
  name = 'Model '+str(model_number+1)
  print("\n\t==>",name)
  generator_train = batch_generator(image_paths_train, batch_size=batch_size, seed=model_number+1)
  generator_val = batch_generator(image_paths_val, batch_size=batch_size, seed=model_number+1)
  base_path = os.path.join('model_data',name)
  if(os.path.exists(base_path)):
    raise NameError('Directory {} already exists. Please delete it first, manually.'.format(base_path))
  os.mkdir(base_path)
  checkpoint = ModelCheckpoint(filepath=os.path.join(base_path,'epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5'),
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            mode='min')
  callbacks = [checkpoint]
  model = get_model(train_last_layers=1)
  model.fit_generator(generator_train,
    epochs=epochs,
    steps_per_epoch=len(image_paths_train)//batch_size,
    validation_data=generator_val,
    validation_steps=len(image_paths_val)//batch_size,
    callbacks=callbacks,
    verbose=1)

# Model Testing & Results

In [0]:
# Load all models for ensembling
ensemble_models = {}
for model_number in range(n_models):
  name = 'Model '+str(model_number+1)
  base_path = os.path.join('model_data',name)
  # Find the last checkpoint for each model
  model_checkpoints = os.listdir(base_path)
  _list = [(int(re.findall("\d{1,3}",model_ckpt)[0]),model_ckpt) for model_ckpt in model_checkpoints]
  if(len(_list)==0):
    raise ValueError('No checkpoints found for {}'.format(name))
  sorted(_list)
  model_file = _list[0][1]
  # Load the model
  model = load_model(os.path.join(base_path,model_file))
  # Append model to the ensemble
  ensemble_models[name] = model

In [0]:
# Test on a random image from validation set
random_choice = list(np.random.choice(image_paths_val,1)) # Choose a random image
test_image, _ = preprocess(random_choice) # Preprocess image
predictions = {}
for model_name,model in ensemble_models.items():
  prediction = model.predict(test_image)
  predictions[model_name] = prediction
print(' > True Age : {}'.format(random_choice[0].split('_')[0]))
ensemble_avg_prediction = 0
for model_name,prediction in predictions.items():
  print(' > Predicted Age by {0} : {1:.4f}'.format(model_name,prediction[0][0]))
  ensemble_avg_prediction = ensemble_avg_prediction + prediction[0][0]
print(' > Predicted Age by Ensembling : {0:.4f}'.format(ensemble_avg_prediction/n_models))
display(Image(filename=os.path.join(imgs_folder_path,random_choice[0]), width=200, height=200))

# References
<hr/>

<ul type="square">
	<li><a href="https://link.springer.com/content/pdf/10.1007%2Fs11263-016-0940-3.pdf">“Deep EXpectation of real and apparent age from a single image without facial landmarks”, 2018 International Journal of Computer Vision 126:144-157</a></li>
</ul>